In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# veri kaynağı: http://veribilimi.co/data/iris.csv
df = pd.read_csv("D:\\OneDrive\\egitim\\verilen\\bogazici-compec-python-ml\\Datasets\\iris.csv")

# 1. Veri Keşfi

In [3]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [5]:
df.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [6]:
# Veri keşfi ile başka birçok groupby, count_values vb. sorgular kullanılabilir. Grafikler çizilip incelenebilr.

# 2. Veri Temizliği ve Ön Hazırlığı

In [7]:
# Veri önceden temizlenmiş olduğundan bu aşamada birşey yapmıyoruz.

## 2.1. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme)

In [8]:
# Hedef değişken en sonda olduğu için ve sadece nümerik nitelikleri kullanacağımız için
# baştan (0) son indekse kadar tüm nitelikleri alıyoruz
X = df.iloc[:,:-1].values
X.shape

(150, 4)

In [9]:
# Kontrol
X[:3]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2]])

In [10]:
# En son indeks hedef değiken (output)
y = df.iloc[:,-1].values
y.shape

(150,)

In [11]:
# Kontrol
y[:10]

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa'], dtype=object)

## 2.2. Kategorik Nitelik Dönüştürme

In [12]:
# Girdi nitelikler içinde kategorik olan yok. Sadece hedef değişken kategorik.

### 2.2.1. Hedef değişken için LabelEncoder

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
# Hedef niteliği sadece LabelEncoder ile rakamlara kodluyoruz.
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)

In [15]:
# Kontrol
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

## 2.3. Feature Scaling (Standardizasyon)

In [16]:
# Nitelikler aynı ölçekte (cm) olduğu için kullanmayabiliriz.

## 2.4. Veriyi eğitim ve test setlerine bölme (yatay bölme)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=142)

In [19]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(112, 4)
(112,)
(38, 4)
(38,)


# 3. Model Oluşturma ve Eğitim

In [20]:
from sklearn.svm import SVC

In [21]:
classifier = SVC(random_state=142, gamma='auto')

In [22]:
classifier.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)

# 4. Predictions

In [23]:
y_pred = classifier.predict(X_test)

## 4.1. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [24]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

,y_pred,y_test
0,0,0
1,1,1
2,1,1
3,2,2
4,1,1


# 5. Model Değerlendirme

## 5.1. Hata Matrisi (Confusion Matrix)

In [25]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(cm)

[[ 7  0  0]
 [ 0 16  1]
 [ 0  1 13]]


## 5.2. Sınıflandırma Performansı Değerlendirme (Accuracy)

In [26]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      94.74


# 6. Hyperparameter Tuning - Model Seçme - En İyi Hiperparametreleri Bulma

In [27]:
# mevcut parametreler
classifier.get_params

<bound method BaseEstimator.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)>

In [28]:
# Lojistik regresyon hiper parametreleri hakkında bilgi için:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

## 6.1. Hiperparametre kombinasyonlarını belirleme

In [28]:
param_grid = [
  {
    'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['linear']
    },
    {
     'C': [1.0, 10.0, 100.0,1000.0], 
   'kernel': ['rbf'],
   'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]
    }
 ]

## 6.2. GridSearchCV ile en iyi hiperparametre setini/modeli bulma

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
grid_search = GridSearchCV(estimator=classifier, 
                           param_grid=param_grid,
                          scoring='accuracy',
                          cv=10,
                          n_jobs=4,
                          iid=False)
# maksimum için n_jobs=-1 kullanılır.
# iid=False deprecation ikazını kapatmak için eklendi

In [31]:
grid_search = grid_search.fit(X_train, y_train)

## 6.3. En iyi skor, hiperparametre ve model

In [32]:
best_score = grid_search.best_score_
best_score

0.990909090909091

In [33]:
best_params = grid_search.best_params_
best_params

{'C': 1.0, 'kernel': 'linear'}

In [34]:
best_estimator = grid_search.best_estimator_
best_estimator

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)

# 7. En iyi Model Performansı

In [35]:
y_pred = best_estimator.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print("Accuracy: % {:10.2f}".format(accuracy*100)) 
# :10.2f ifadesi ondalık kısımda iki basamak göstermek için

Accuracy: %      94.74


In [37]:
# Değişen birşey olmadı. Zaten baz sınıflandırıcı başarısı yüksekti, 2 hata yapmıştı.
# Yukarıdaki best_score'dan farklı olmasının sebebi o train setinde bu test setinde hesaplandı

# 8. Modeli Saklama ve Tekrar Kullanma

In [36]:
# Saklamadan önce tüm veri seti ile modeli en iyi hiperparametrelerle tekrar eğitelim

## 8.1. Tüm veri ile modeli son olarak eğitme

In [39]:
# Boş bir sınıflandırıcı (SVC) nesnesi oluşturalım
classifier_last = SVC(random_state=142, gamma='auto')

In [40]:
# Boş nesneye en iyi modelin hiperparametrelerini aktaralım
classifier_last.set_params = best_estimator.get_params

In [41]:
# En iyi hiperparametrelerle sınıflandırıcıyı/estimator/modeli eğitelim
classifier_last.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)

## 8.2. Modeli kaydetme

In [43]:
from sklearn.externals import joblib

In [44]:
joblib.dump(classifier_last, 
"D:/OneDrive/egitim/verilen/bogazici-compec-python-ml/08_classification/saved_models/05.Model_selection_tuning_with_iris_data.pkl")

['05.Model_selection_tuning_with_iris_data.pkl']

## 8.3. Model Okuma

In [45]:
classifier_last_loaded = joblib.load("D:/OneDrive/egitim/verilen/bogazici-compec-python-ml/08_classification/saved_models/05.Model_selection_tuning_with_iris_data.pkl")

In [46]:
classifier_last_loaded.get_params

<bound method BaseEstimator.get_params of SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=142, shrinking=True,
  tol=0.001, verbose=False)>